<a href="https://colab.research.google.com/github/Deepsaha07/PPG_Analysis/blob/master/PPG_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [4]:
import numpy as np
import sys
from sklearn.preprocessing import StandardScaler
#np.set_printoptions(threshold=sys.maxsize)
from keras.models import Sequential

from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Embedding
from keras.layers import Conv1D, MaxPooling1D, Activation, Flatten
from keras.preprocessing import sequence
import scipy.io as scio
# fix random seed for reproducibility
np.random.seed(7)
import warnings
warnings.filterwarnings('ignore')

In [5]:
import tensorflow
import sys
from sklearn.preprocessing import StandardScaler
#np.set_printoptions(threshold=sys.maxsize)
from keras.models import Sequential
from numba import jit,cuda
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Embedding
from keras.layers import Conv1D, MaxPooling1D, Activation, Flatten
from keras.preprocessing import sequence
import scipy.io as scio
# fix random seed for reproducibility
np.random.seed(7)
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import os
import pandas as pd

print(os.getcwd())
path = '/content/gdrive/MyDrive/csv'

set = {}


for dataset in os.listdir(path):

     head,sep,tail = dataset.partition('.')
     #print(head)
     data = pd.read_csv(os.path.join(path,dataset))
     set[head] = data








/content


In [8]:

alllabels = pd.DataFrame()
dset = pd.DataFrame()
i=0
for key,value in set.items():
    #subs.append(i.split('_')
    subs=[]
    labels = []
    featureset=[]
    data = pd.DataFrame(value.iloc[:,:9])
    sub_label = key.split('_')
    sub = sub_label[0]
    label = sub_label[1]
    for i in range(len(data)):
        subs.append([sub])
        labels.append(label)
    labels = pd.DataFrame(labels,columns=['Label'])
    subs = pd.DataFrame(subs,columns=['SubjectID'])
    featureset = pd.concat([subs,data,labels],axis=1)
    if i<1:
        dset = featureset
        #alllabels = labels
    else:
        
        f = featureset.copy()
        #l = labels.copy()
        dset = pd.concat([dset,f],axis=0)
        #alllabels = pd.concat([alllabels,l],axis=0)
    i+=1
    

In [9]:
dset=dset.drop(['time'],axis=1)

In [10]:
features = dset.drop(['SubjectID','Label'],axis =1)
featureshape = features.shape[1]

In [11]:
@jit(target_backend='cuda')
def olrem(data):
    data1 = data.copy()
    for key,item in data.items():
        
    
        Q1 = item.quantile(0.25)
        Q3 = item.quantile(0.75)
        IQR = Q3 - Q1
        lrange = Q1 - 1.5*IQR
        urange = Q3 + 1.5*IQR

        data1.loc[data1[key]<lrange,key]=0
        data1.loc[data1[key]>urange,key]=0
        
        data.loc[data[key]<lrange,key] = np.mean(data1[key])
        data.loc[data[key]>urange,key]= np.mean(data1[key])
    
        
        

        return data  

In [12]:
refet = olrem(features)

In [13]:
scale = StandardScaler()
fet = scale.fit_transform(refet)
fet.astype(np.float16)
dsett= dset.copy()
dsett.iloc[:,1:-1] = fet

In [15]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
le = LabelEncoder()
enc = OrdinalEncoder()
dsett['Label']=le.fit_transform(dsett['Label'])
dsett['SubjectID'] = le.fit_transform(dsett['SubjectID'])

In [17]:
grouped = dsett.groupby('SubjectID')

In [18]:
from keras.layers import Dropout
model = Sequential()
#model.add(Embedding(tot_length, embedding_vector_length, input_length=max_length))
#model.add(Dropout(0.2))
model.add(LSTM(50,activation = 'tanh',return_sequences=True, input_shape = (1,featureshape)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(80,activation = 'sigmoid',return_sequences=True, stateful=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.build()
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             11800     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 batch_normalization (BatchN  (None, 1, 50)            200       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 1, 80)             41920     
                                                                 
 dropout_1 (Dropout)         (None, 1, 80)             0         
                                                                 
 dense (Dense)               (None, 1, 1)              81        
                                                        

In [24]:
with tf.device('/device:GPU:0'):
    

      for group in grouped:
          
          batch = pd.DataFrame(columns = dsett.columns)
          for subjects in group:
              
              filter=dsett['SubjectID']==subjects
              for key,item in dsett[filter].dropna().iterrows():
                  
                  batch = batch.append(item)
          X= batch[features.columns]
          X= np.resize(X,(X.shape[0],1,X.shape[1]))
          y = batch['label']
          model.fit(X,y,batch_size=len(X),epochs = 10)

KeyboardInterrupt: ignored

In [22]:
model_fit(grouped,dsett,features)

KeyboardInterrupt: ignored

In [ ]:
dsett.columns